In [1]:
using BenchmarkTools

struct Thing
    somevalue::Int
end

things = [Thing(rand(1:5)) for _ = 1:100000]

threes_getprop = Iterators.filter(a -> getproperty(a, :somevalue) === 3, things)
threes_prop = Iterators.filter(a -> a.somevalue === 3, things)

conditional_property_iterator(iterable, property::Symbol, value) = Iterators.filter(a -> getproperty(a, property) === value, iterable)
threes_cond = conditional_property_iterator(things, :somevalue, 3)

function test_threes_getproperty()
    for i in threes_getprop
        i.somevalue + rand(Int)
    end
end

function test_threes_prop()
    for i in threes_prop
        i.somevalue + rand(Int)
    end
end

function test_threes_cond()
    for i in threes_cond
        i.somevalue + rand(Int)
    end
end

@btime test_threes_getproperty()
@btime test_threes_prop()
@btime test_threes_cond()

  2.588 ms (120571 allocations: 2.15 MiB)
  2.571 ms (120571 allocations: 2.15 MiB)
  4.219 ms (220571 allocations: 3.67 MiB)


In [14]:
function test_threes(threes_itr)
    for i in threes_itr
        i.somevalue + rand(Int)
    end
end

@btime test_threes($threes_getprop)
@btime test_threes($threes_prop)
@btime test_threes($threes_cond)

  174.300 μs (0 allocations: 0 bytes)
  169.700 μs (0 allocations: 0 bytes)
  1.650 ms (100000 allocations: 1.53 MiB)


In [15]:
conditional_property_iterator2(iterable, property, value) = Iterators.filter(Base.Fix2(===, value)∘Base.Fix2(getproperty, property), iterable)
threes_cond2 = conditional_property_iterator2(things, :somevalue, 3)

Base.Iterators.Filter{ComposedFunction{Base.Fix2{typeof(===), Int64}, Base.Fix2{typeof(getproperty), Symbol}}, Vector{Thing}}(Base.Fix2{typeof(===), Int64}(===, 3) ∘ Base.Fix2{typeof(getproperty), Symbol}(getproperty, :somevalue), Thing[Thing(3), Thing(3), Thing(1), Thing(4), Thing(1), Thing(1), Thing(3), Thing(3), Thing(3), Thing(2)  …  Thing(4), Thing(1), Thing(4), Thing(4), Thing(2), Thing(1), Thing(2), Thing(5), Thing(3), Thing(1)])

In [16]:
@btime test_threes($threes_cond2)

  1.652 ms (100000 allocations: 1.53 MiB)


In [22]:
@btime collect($threes_getprop)
@btime collect($threes_prop)
@btime collect($threes_cond)

  288.900 μs (9 allocations: 326.55 KiB)
  293.100 μs (9 allocations: 326.55 KiB)
  1.759 ms (100009 allocations: 1.84 MiB)


20112-element Vector{Thing}:
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 ⋮
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)

In [23]:
@code_warntype collect(threes_getprop)

MethodInstance for collect(::Base.Iterators.Filter{var"#3#4", Vector{Thing}})
  from collect(itr) in Base at array.jl:694
Arguments
  #self#::Core.Const(collect)
  itr::Base.Iterators.Filter{var"#3#4", Vector{Thing}}
Body::Vector{Thing}
1 ─ %1 = (1:1)::Core.Const(1:1)
│   %2 = Base.IteratorEltype(itr)::Core.Const(Base.HasEltype())
│   %3 = Base.IteratorSize(itr)::Core.Const(Base.SizeUnknown())
│   %4 = Base._collect(%1, itr, %2, %3)::Vector{Thing}
└──      return %4



In [34]:
@code_warntype Base._similar_for(1:1, Thing, threes_prop, Base.SizeUnknown())

MethodInstance for Base._similar_for(::UnitRange{Int64}, ::Type{Thing}, ::Base.Iterators.Filter{var"#5#6", Vector{Thing}}, ::Base.SizeUnknown)
  from _similar_for(c::AbstractArray, ::Type{T}, itr, ::Base.SizeUnknown) where T in Base at array.jl:657
Static Parameters
  T = Thing
Arguments
  #self#::Core.Const(Base._similar_for)
  c::UnitRange{Int64}
  _::Core.Const(Thing)
  itr::Base.Iterators.Filter{var"#5#6", Vector{Thing}}
  _::Core.Const(Base.SizeUnknown())
Body::Vector{Thing}
1 ─ %1 = Base.similar(c, $(Expr(:static_parameter, 1)), 0)::Vector{Thing}
└──      return %1



In [35]:
@code_warntype Base._similar_for(1:1, Thing, threes_cond, Base.SizeUnknown())

MethodInstance for Base._similar_for(::UnitRange{Int64}, ::Type{Thing}, ::Base.Iterators.Filter{var"#7#8"{Symbol, Int64}, Vector{Thing}}, ::Base.SizeUnknown)
  from _similar_for(c::AbstractArray, ::Type{T}, itr, ::Base.SizeUnknown) where T in Base at array.jl:657
Static Parameters
  T = Thing
Arguments
  #self#::Core.Const(Base._similar_for)
  c::UnitRange{Int64}
  _::Core.Const(Thing)
  itr::Base.Iterators.Filter{var"#7#8"{Symbol, Int64}, Vector{Thing}}
  _::Core.Const(Base.SizeUnknown())
Body::Vector{Thing}
1 ─ %1 = Base.similar(c, $(Expr(:static_parameter, 1)), 0)::Vector{Thing}
└──      return %1



In [37]:
Base._similar_for(1:1, Thing, threes_cond, Base.SizeUnknown())

Thing[]

In [36]:
Base._collect(1:1, threes_cond, Base.HasEltype(), Base.SizeUnknown())

20112-element Vector{Thing}:
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 ⋮
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)
 Thing(3)

In [27]:
@code_warntype Base._collect(1:1, threes_cond, Base.HasEltype(), Base.SizeUnknown())

MethodInstance for Base._collect(::UnitRange{Int64}, ::Base.Iterators.Filter{var"#7#8"{Symbol, Int64}, Vector{Thing}}, ::Base.HasEltype, ::Base.SizeUnknown)
  from _collect(cont, itr, ::Base.HasEltype, isz::Base.SizeUnknown) in Base at array.jl:703
Arguments
  #self#::Core.Const(Base._collect)
  cont::UnitRange{Int64}
  itr::Base.Iterators.Filter{var"#7#8"{Symbol, Int64}, Vector{Thing}}
  _::Core.Const(Base.HasEltype())
  isz::Core.Const(Base.SizeUnknown())
Locals
  @_6::Union{Nothing, Tuple{Thing, Int64}}
  a::Vector{Thing}
  x::Thing
Body::Vector{Thing}
1 ─ %1  = Base.eltype(itr)::Core.Const(Thing)
│         (a = Base._similar_for(cont, %1, itr, isz))
│   %3  = itr::Base.Iterators.Filter{var"#7#8"{Symbol, Int64}, Vector{Thing}}
│         (@_6 = Base.iterate(%3))
│   %5  = (@_6 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_6::Tuple{Thing, Int64}
│         (x = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│         Base.push!

In [39]:
@code_warntype iterate(threes_cond, 1)

MethodInstance for iterate(::Base.Iterators.Filter{var"#7#8"{Symbol, Int64}, Vector{Thing}}, ::Int64)
  from iterate(f::Base.Iterators.Filter, state...) in Base.Iterators at iterators.jl:444
Arguments
  #self#::Core.Const(iterate)
  f::Base.Iterators.Filter{var"#7#8"{Symbol, Int64}, Vector{Thing}}
  state::Tuple{Int64}
Locals
  y::Union{Nothing, Tuple{Thing, Int64}}
Body::Union{Nothing, Tuple{Thing, Int64}}
1 ─ %1  = Base.getproperty(f, :itr)::Vector{Thing}
│   %2  = Core.tuple(%1)::Tuple{Vector{Thing}}
└──       (y = Core._apply_iterate(Base.iterate, Base.Iterators.iterate, %2, state))
2 ┄ %4  = (y !== Base.Iterators.nothing)::Bool
└──       goto #6 if not %4
3 ─ %6  = Base.getproperty(f, :flt)::var"#7#8"{Symbol, Int64}
│   %7  = Base.getindex(y::Tuple{Thing, Int64}, 1)::Thing
│   %8  = (%6)(%7)::Bool
└──       goto #5 if not %8
4 ─       return y::Tuple{Thing, Int64}
5 ─ %11 = Base.getproperty(f, :itr)::Vector{Thing}
│   %12 = Base.getindex(y::Tuple{Thing, Int64}, 2)::Int64
│        

In [29]:
threes_getprop |> eltype

Thing

In [30]:
threes_prop |> eltype

Thing

In [31]:
threes_cond |> eltype

Thing

In [32]:
threes_cond.flt |> dump

#7 (function of type var"#7#8"{Symbol, Int64})
  property: Symbol somevalue
  value: Int64 3


In [33]:
threes_cond2.flt |> dump

ComposedFunction{Base.Fix2{typeof(===), Int64}, Base.Fix2{typeof(getproperty), Symbol}}(Base.Fix2{typeof(===), Int64}(===, 3), Base.Fix2{typeof(getproperty), Symbol}(getproperty, :somevalue)) (function of type ComposedFunction{Base.Fix2{typeof(===), Int64}, Base.Fix2{typeof(getproperty), Symbol}})
  outer: Base.Fix2{typeof(===), Int64}(===, 3) (function of type Base.Fix2{typeof(===), Int64})
    f: === (function of type typeof(===))
    x: Int64 3
  inner: Base.Fix2{typeof(getproperty), Symbol}(getproperty, :somevalue) (function of type Base.Fix2{typeof(getproperty), Symbol})
    f: getproperty (function of type typeof(getproperty))
    x: Symbol somevalue
